In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout, HTML

from IPython.core.interactiveshell import InteractiveShell
from ipyevents import Event
import random

In [2]:
# Setup global configurations
#InteractiveShell.ast_node_interactivity = "all"
max_rows = 20
pd.set_option('display.max_rows', max_rows)

In [3]:
# Load empty board
df = pd.read_csv('board.csv', header=None)
index = []
for i in range(-4, 21):
    index.append(i)
df.set_index([pd.Index(index)], inplace=True)

In [4]:
# Initialize board
output = widgets.Output()
board = widgets.HBox([output], layout=Layout(width='400px', height='600px'))
def refreshBoard(source=None):
    output.clear_output(wait=True)
    with output:
        display(df.iloc[4:24])

In [5]:
# Drop new tetromino
t = {}
def randomTetromino():
    global t
    tetrominos = [{'type':'I', 'location':[(0,4), (1, 4), (2,4), (3,4)]}, 
                  {'type':'O', 'location':[(2,4), (2, 5), (3,4), (3,5)]},
                  {'type':'T', 'location':[(2,5), (3, 4), (3,5), (3,6)]},
                  {'type':'S', 'location':[(2,5), (2, 6), (3,4), (3,5)]},
                  {'type':'Z', 'location':[(2,4), (2, 5), (3,4), (3,5)]},
                  {'type':'J', 'location':[(2,4), (3, 4), (3,5), (3,6)]},
                  {'type':'L', 'location':[(2,6), (3, 4), (3,5), (3,6)]}]
    t = random.choice(tetrominos)
    for l in t['location']:
        df.iloc[l] = "#"

In [6]:
def move(dir): 
    nt = [] 
    for i in range(0, 4):
        if dir == "down":
            nt.append((t['location'][i][0]+1, t['location'][i][1]))
        elif dir == "left":
            nt.append((t['location'][i][0], t['location'][i][1]-1))
        elif dir == "right":
            nt.append((t['location'][i][0], t['location'][i][1]+1))
            
    blocked = False
    # make sure nothing blocks
    for i in range(0, 4):
        if df.iloc[nt[i]] == "#" and nt[i] not in t['location']:
            blocked = True
            if dir == "down":
                randomTetromino()
            break
        if dir == "left" and nt[i][1] < 0:
            blocked = True
            break
        if dir == "right" and nt[i][1] > 10:
            blocked = True
            break
    if not blocked:
        for l in t['location']:
            df.iloc[l] = ' '
        for i in range(0, 4):
            t['location'][i] = nt[i]
        for l in t['location']:
            df.iloc[l] = "#"

In [7]:
def moveDown(): 
    nt = [] 
    for i in range(0, 4):
        nt.append((t['location'][i][0]+1, t['location'][i][1]))
    blocked = False
    # make sure nothing blocks
    for i in range(0, 4):
        if df.iloc[nt[i]] == "#" and nt[i] not in t['location']:
            blocked = True
            randomTetromino()
            break
    if not blocked:
        for l in t['location']:
            df.iloc[l] = ' '
        for i in range(0, 4):
            t['location'][i] = nt[i]
        for l in t['location']:
            df.iloc[l] = "#"

In [8]:
# Listen to keyboard events
keyPressed = HTML('Key Pressed')
boardEvent = Event(source=output, watched_events=['keydown'])
def handle_event(event):
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    keyPressed.value = event['code']
    if event['code'] == 'ArrowDown':
        move("down")
    elif event['code'] == 'ArrowLeft':
        move("left")
    elif event['code'] == 'ArrowRight':
        move("right")
    refreshBoard(None)
boardEvent.on_dom_event(handle_event)

In [9]:
# Start game 
def startGame():
    randomTetromino()
    display(board, keyPressed)
    refreshBoard()

In [10]:
startGame()

HTML(value='Key Pressed')